In [1]:
import pandas as pd
import requests
import zipfile
import os

In [2]:
diretorio_atual = os.getcwd()

os.makedirs('/content/dados_cvm')
os.chdir('/content/dados_cvm')

In [3]:
anos = range(2010, 2022)

url_base_dfp = 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/'
# url_base_itr = 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/'

In [4]:
for ano in anos:
  download = requests.get(url_base_dfp + f'dfp_cia_aberta_{ano}.zip')
  open(f'dfp_cia_aberta_{ano}.zip', 'wb').write(download.content)

In [3]:
diretorio_atual = os.getcwd()
lista_demostrativos = []

In [4]:
for arquivo in os.listdir(diretorio_atual):
  if arquivo == '.ipynb_checkpoints':
    pass
  else:
    arquivo_zip = zipfile.ZipFile(arquivo)
    for planilha in arquivo_zip.namelist():
      demonstracao = pd.read_csv(arquivo_zip.open(planilha), sep = ';', encoding = 'ISO-8859-1', dtype = {'ORDEM_EXERC' : 'category'})
      lista_demostrativos.append(demonstracao)

In [5]:
base_dados = pd.concat(lista_demostrativos)
base_dados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12083995 entries, 0 to 54712
Data columns (total 20 columns):
 #   Column         Dtype   
---  ------         -----   
 0   CNPJ_CIA       object  
 1   DT_REFER       object  
 2   VERSAO         int64   
 3   DENOM_CIA      object  
 4   CD_CVM         int64   
 5   CATEG_DOC      object  
 6   ID_DOC         float64 
 7   DT_RECEB       object  
 8   LINK_DOC       object  
 9   GRUPO_DFP      object  
 10  MOEDA          object  
 11  ESCALA_MOEDA   object  
 12  ORDEM_EXERC    category
 13  DT_FIM_EXERC   object  
 14  CD_CONTA       object  
 15  DS_CONTA       object  
 16  VL_CONTA       float64 
 17  ST_CONTA_FIXA  object  
 18  DT_INI_EXERC   object  
 19  COLUNA_DF      object  
dtypes: category(1), float64(2), int64(2), object(15)
memory usage: 1.8+ GB


In [6]:
base_dados[['con_ind', 'tipo_dem']] = base_dados['GRUPO_DFP'].str.split('-', expand = True)
base_dados['con_ind'] = base_dados['con_ind'].str.strip()
base_dados['tipo_dem'] = base_dados['tipo_dem'].str.strip()

base_dados = base_dados[base_dados['ORDEM_EXERC'] != 'PENÚLTIMO']

In [7]:
base_dados.head()

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,CATEG_DOC,ID_DOC,DT_RECEB,LINK_DOC,GRUPO_DFP,...,ORDEM_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA,DT_INI_EXERC,COLUNA_DF,con_ind,tipo_dem
0,00.000.000/0001-91,2014-12-31,1,BCO BRASIL S.A.,1023,DFP,43994.0,2015-02-11,http://www.rad.cvm.gov.br/ENETCONSULTA/frmDown...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00.000.000/0001-91,2014-12-31,2,BCO BRASIL S.A.,1023,DFP,45354.0,2015-03-27,http://www.rad.cvm.gov.br/ENETCONSULTA/frmDown...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00.000.208/0001-00,2014-12-31,1,BRB BCO DE BRASILIA S.A.,14206,DFP,45265.0,2015-03-25,http://www.rad.cvm.gov.br/ENETCONSULTA/frmDown...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00.000.208/0001-00,2014-12-31,2,BRB BCO DE BRASILIA S.A.,14206,DFP,46034.0,2015-04-17,http://www.rad.cvm.gov.br/ENETCONSULTA/frmDown...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00.001.180/0001-26,2014-12-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DFP,45472.0,2015-03-27,http://www.rad.cvm.gov.br/ENETCONSULTA/frmDown...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
base_dados.to_csv('/content/drive/MyDrive/Datasets Finance/cvm_dados.csv', sep = ';', decimal = '.', encoding = 'ISO-8859-1', index = False)